SVM using BUG dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from lime import lime_text
from lime.lime_text import LimeTextExplainer
import pickle
import xgboost as xgb

In [2]:
# load data
dataset = pd.read_csv('../datasets/BUG/balanced_BUG.csv')
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
# get samples with only neutral or stereotype sentence
train = train[train['stereotype'].isin([0, 1])]
test = test[test['stereotype'].isin([0, 1])]

vectorizer = CountVectorizer()
trainTexts = vectorizer.fit_transform(train['sentence_text'])
testTexts = vectorizer.transform(test['sentence_text'])

xTrain = pd.DataFrame(trainTexts.toarray(), columns=vectorizer.get_feature_names_out())
xTest = pd.DataFrame(testTexts.toarray(), columns=vectorizer.get_feature_names_out())
yTrain = train['predicted gender']
yTest = test['predicted gender']

# train model
model = LinearSVC()
model.fit(xTrain, yTrain)

model = CalibratedClassifierCV(model, method='sigmoid', cv='prefit')
model.fit(xTrain, yTrain)

with open('../savedModels/svmModel.pkl', 'wb') as model_file:
    pickle.dump((model, vectorizer), model_file)

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [3]:
# accuracy
pred = model.predict(xTest)

accuracy = accuracy_score(yTest, pred)
print("Accuracy:", accuracy)

# confusion matrix
cm = confusion_matrix(yTest, pred)
print("Confusion matrix:\n", cm)

Accuracy: 0.9875097580015613
Confusion matrix:
 [[1280   15]
 [  17 1250]]


In [4]:
text = [
"""
Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hands in the soil and a heart full of passion , Mia transformed an unused corner of the campus into a thriving community garden . She envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate their own fruits and vegetables . Mia believed that this initiative could not only promote environmental consciousness but also foster a sense of community among her peers . As the garden flourished , so did Mia 's connection with her fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables . Mia 's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived , Mia nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of her project resonated deeply , earning her the first prize and a scholarship for further environmental studies . Word of Mia 's success spread , and GreenHaven became a symbol of sustainable living on campus . Mia 's journey didn 't end with the competition ; instead , it marked the beginning of a new chapter . With the scholarship in hand , Mia continued her studies , conducting research on innovative ways to create sustainable urban environments . As Mia graduated from Rivertide University , she left behind a legacy of green initiatives and a campus that had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues . Mia 's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as Mia embarked on her journey beyond college , she carried with her not just a degree but the knowledge that small , meaningful actions could ripple into waves of positive transformation for the planet and its people .
""",
"""
Once upon a time in the vibrant city of Harmonyville ,  lived a college student named   .  was a junior majoring in environmental science at Rivertide University .  love for nature and a deep sense of responsibility towards the planet fueled  determination to make a positive impact . One sunny afternoon ,  stumbled upon a notice about the annual Green Innovation Challenge — an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute ,  decided to develop a sustainable urban gardening initiative called   " GreenHaven . " With  hands in the soil and a heart full of passion ,  transformed an unused corner of the campus into a thriving community garden .  envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate  own fruits and vegetables .  believed that this initiative could not only promote environmental consciousness but also foster a sense of community among  peers . As the garden flourished , so did  's connection with  fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables .  's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived ,  nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of  project resonated deeply , earning  the first prize and a scholarship for further environmental studies . Word of  's success spread , and  became a symbol of sustainable living on campus .  's    t end with the competition ; instead ,  marked the beginning of a new chapter . With the scholarship in hand ,  continued  studies , conducting research on innovative ways to create sustainable urban environments . As  graduated from Rivertide University ,  left behind a legacy of green initiatives and a campus  had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues .  's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as  embarked on  journey beyond college ,  carried with  not just a degree but the knowledge  small , meaningful actions could ripple into waves of positive transformation for the planet and  people .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student named Alex Reynolds . Alex was a junior majoring in computer science at the prestigious Arcadia University . He was a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as Alex was immersed in his studies at the campus library , he stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student . This student was a junior majoring in computer science at the prestigious Arcadia University . They were a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as this student was immersed in their studies at the campus library , they stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the vibrant college town of Crestwood , there lived a spirited young student named Emily . She was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for her boundless enthusiasm , infectious energy , and a love for exploring the world around her . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called  "Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes , she decided to attend their next meeting . As Emily walked into the meeting room , she was greeted by a diverse group of students who shared her passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the Green Harmony team worked tirelessly , planning , planting , and nurturing their garden . Emily found herself forming deep connections with her fellow club members as they faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did Emily 's personal growth . She learned about sustainable practices , organic gardening , and the importance of community engagement . Her once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded them a grant to expand their project . Emily and her friends found themselves at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only her passion for environmental science but also her ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for those around her . As the seasons changed , so did Emily and her friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student who discovered her purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
Once upon a time in the vibrant college town of Crestwood ,  lived a spirited young student named  .  was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for  boundless enthusiasm , infectious energy , and a love for exploring the world around  . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called   " Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes ,  decided to attend  next meeting . As Emily walked into the meeting room ,  was greeted by a diverse group of students  shared  passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the   team worked tirelessly , planning , planting , and nurturing  garden . Emily found  forming deep connections with  fellow club members as  faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did  's personal growth .  learned about sustainable practices , organic gardening , and the importance of community engagement .  once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded  a grant to expand  project . Emily and  friends found  at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only  passion for environmental science but also  ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for  around  . As the seasons changed , so did  and  friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student  discovered  purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
In the vibrant city of Rivertown , there lived a college student named Jason Harris . Jason was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller . He had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper , Jason stumbled upon an article about an elderly woman named Mrs . Eleanor Bennett , who had spent decades working as a librarian in the city . Intrigued by the snippet of her life story , Jason felt compelled to dig deeper and share her tale with the world . After some research , Jason learned that Mrs . Bennett had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However , her own life story had largely remained in the shadows . Determined to uncover the full narrative , Jason reached out to Mrs . Bennett and asked for an interview . To his surprise , she welcomed the opportunity to share her story , and soon they sat down for a conversation that would unveil a lifetime of experiences . As Jason delved into Mrs . Bennett 's past , he discovered a woman who had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown . Her journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative that Jason couldn 't wait to share with his readers . With meticulous attention to detail , Jason crafted a feature article that not only highlighted Mrs . Bennett 's contributions to the community but also captured the essence of her character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs . Bennett became a beloved figure in Rivertown , celebrated for her dedication and the richness of her life story . The success of the article opened doors for Jason , who found himself on the path to becoming a respected journalist . Inspired by Mrs . Bennett 's story , he continued to seek out and share the untold narratives within his community , becoming a storyteller who bridged the gaps between generations . As graduation day approached , Jason reflected on his college journey . His time at Rivertown University had not only provided him with an education but had also instilled in him the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories , Jason ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the vibrant city of Rivertown ,  lived a college student named   .  was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller .  had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper ,  stumbled upon an article about an elderly woman named  .   ,  had spent decades working as a librarian in the city . Intrigued by the snippet of  life story ,  felt compelled to dig deeper and share  tale with the world . After some research ,  learned that  .  had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However ,  own life story had largely remained in the shadows . Determined to uncover the full narrative ,  reached out to  .  and asked for an interview . To  surprise ,  welcomed the opportunity to share  story , and soon  sat down for a conversation  would unveil a lifetime of experiences . As  delved into  .  's past ,  discovered a woman  had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown .  journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative   couldn ' t wait to share with  readers . With meticulous attention to detail ,  crafted a feature article that not only highlighted  .  's contributions to the community but also captured the essence of  character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs .  became a beloved figure in Rivertown , celebrated for  dedication and the richness of  life story . The success of the article opened doors for  ,  found  on the path to becoming a respected journalist . Inspired by  .  's story ,  continued to seek out and share the untold narratives within  community , becoming a storyteller  bridged the gaps between generations . As graduation day approached ,  reflected on  college journey .  time at Rivertown University had not only provided  with an education but had also instilled in  the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories ,  ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the bustling town of Summitville , there lived a college man named Lucas Turner . Lucas was a mechanical engineering major at Summitville Tech , known for his passion for building things from the ground up . His college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with his fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in him . Inspired by the potential of the neglected space , he decided to breathe new life into it and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together they transformed the workshop into the  "Innovation Forge . " It became a haven for students to work on personal projects , exchange ideas , and bring their engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on everything from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and his team decided to organize an annual engineering expo , showcasing the innovative projects that emerged from their workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and his team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for its engineering prowess but also for its potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects that aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ; it was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy he had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner . His college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
""",
"""
In the bustling town of Summitville ,  lived a college man named   . Lucas was a mechanical engineering major at Summitville Tech , known for  passion for building things from the ground up .  college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with  fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in  . Inspired by the potential of the neglected space ,  decided to breathe new life into  and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together  transformed the workshop into the   " Innovation Forge . "  became a haven for students to work on personal projects , exchange ideas , and bring  engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on  from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and  team decided to organize an annual engineering expo , showcasing the innovative projects  emerged from  workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and  team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for  engineering prowess but also for  potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects  aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ;  was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy  had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner .  college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
"""
]
vectorizedText = vectorizer.transform(text)
textsTransformed = pd.DataFrame(vectorizedText.toarray(), columns=vectorizer.get_feature_names_out())
predProb = model.predict_proba(textsTransformed)
pred = model.predict(textsTransformed)

yActual = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1]
explainer = LimeTextExplainer(class_names=['female', 'male'])
for i in range(len(pred)):
    print(i + 1, ":")
    print("Male: ", predProb[i][1])
    print("Female: ", predProb[i][0])
    print("Predicted: ", pred[i])
    if(yActual[i] == 0):
        print("Actual: Female")
    else:
        print("Actual: Male")
    
    vectorized_text = vectorizer.transform([text[i]])
    predict_function = lambda x: model.predict_proba(vectorizer.transform(x))
    explanation = explainer.explain_instance(text[i], predict_function, num_features=20)
    top_words_lime = explanation.as_list()
    print(f"Top words for text response {i + 1}:")
    masculineWords = []
    feminineWords = []
    for word, score in top_words_lime:
        if score > 0:
            masculineWords.append((word, score))
        else:
            feminineWords.append((word, score))
    print("Masculine words: ", masculineWords)
    print("Feminine words: ", feminineWords)
    print("")





1 :
Male:  4.583630904533539e-37
Female:  1.0
Predicted:  female
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 1:
Masculine words:  [('of', 0.023217969789085753), ('the', 0.019850307561715318), ('into', 0.014196773174421385), ('come', 0.013298688040336184), ('annual', 0.01128686921851896), ('for', 0.010740834773249457), ('college', 0.010092206558169127)]
Feminine words:  [('her', -0.049761836092567416), ('student', -0.0493513904693281), ('she', -0.04434976929837419), ('Her', -0.034600807530578655), ('She', -0.028329666142787365), ('about', -0.019190889298751665), ('students', -0.01430262699923828), ('even', -0.013889061136720601), ('environmental', -0.0114408039270223), ('presented', -0.010919434676077241), ('ways', -0.010792720034731168), ('in', -0.01009727406143404), ('further', -0.009819130552230857)]

2 :
Male:  3.3466445295315325e-05
Female:  0.9999665335547047
Predicted:  female
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 2:
Masculine words:  [('of', 0.1280764356619574), ('the', 0.1260181125460982), ('to', 0.0442396371022015), ('come', 0.04258245283517468), ('on', 0.03534272457246779), ('Once', 0.0287345190434594), ('and', 0.028669295183162868), ('college', 0.02762755253486234), ('for', 0.02564494968232141), ('believed', 0.020100544252393025), ('hand', 0.01995646737544452), ('University', 0.014420200760090445)]
Feminine words:  [('student', -0.547824948906003), ('about', -0.11408751148919734), ('Green', -0.08449475327364127), ('could', -0.08105097437712823), ('living', -0.07568362752551554), ('students', -0.06938110480160713), ('own', -0.06615379012762328), ('green', -0.0532142755568898)]

3 :
Male:  0.7026919248084144
Female:  0.29730807519158564
Predicted:  male
Actual: Male
Top words for text response 3:
Masculine words:  [('his', 0.2881208303322273), ('He', 0.24088615518721743), ('he', 0.23916351192604715), ('Once', 0.08693641377511749), ('the', 0.07285209557363535), ('fo

...

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 8:
Masculine words:  [('the', 0.14195139529379247), ('had', 0.10589982840399097), ('life', 0.1046068246189731), ('of', 0.06712037208957533), ('for', 0.061485473496906536), ('to', 0.055476036486825316), ('full', 0.04367436811782674), ('an', 0.043025882650115845), ('surprise', 0.0331045486848779)]
Feminine words:  [('librarian', -0.22859194502467473), ('student', -0.20021870396936128), ('journalist', -0.16592189654655246), ('story', -0.12746155547759488), ('Mrs', -0.09234775224741149), ('out', -0.09113059345951682), ('in', -0.09019488280259902), ('only', -0.06731385013842194), ('at', -0.0607468123675423), ('lived', -0.05094098540984345), ('own', -0.04989995112437865)]

9 :
Male:  1.0
Female:  0.0
Predicted:  male
Actual: Male


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 9:
Masculine words:  [('his', 0.029481400240510762), ('he', 0.027445548532937448), ('His', 0.02600490099409685), ('the', 0.02199464338679458), ('on', 0.01364873379012274), ('of', 0.01154050899611273), ('life', 0.011493857888772012), ('for', 0.011059610715832663), ('to', 0.010673786211852663), ('decided', 0.010011736173122916), ('continued', 0.008769454604444076)]
Feminine words:  [('became', -0.011923760525352383), ('One', -0.01122766645307607), ('help', -0.011059261558027935), ('night', -0.009941494403473247), ('personal', -0.009701866108560856), ('lived', -0.009228522992420083), ('was', -0.00808221358088134), ('impact', -0.0065059713486025185), ('It', -0.005677575532789112)]

10 :
Male:  0.9999548171668828
Female:  4.518283311716331e-05
Predicted:  male
Actual: Male
Top words for text response 10:
Masculine words:  [('the', 0.14849342302510596), ('life', 0.11313891627732289), ('of', 0.07510277270535512), ('for', 0.0727544694787192), ('on', 0.07220600122939

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Random Forest Using BUG Dataset

In [5]:
# load data
dataset = pd.read_csv('../datasets/BUG/balanced_BUG.csv')
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
# get samples with only neutral or stereotype sentence
train = train[train['stereotype'].isin([0, 1])]
test = test[test['stereotype'].isin([0, 1])]

vectorizer = CountVectorizer()
trainTexts = vectorizer.fit_transform(train['sentence_text'])
testTexts = vectorizer.transform(test['sentence_text'])

xTrain = pd.DataFrame(trainTexts.toarray(), columns=vectorizer.get_feature_names_out())
xTest = pd.DataFrame(testTexts.toarray(), columns=vectorizer.get_feature_names_out())
yTrain = train['predicted gender']
yTest = test['predicted gender']

# train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(xTrain, yTrain)

with open('../savedModels/randomForestModel.pkl', 'wb') as model_file:
    pickle.dump((model, vectorizer), model_file)

In [6]:
# accuracy
pred = model.predict(xTest)

accuracy = accuracy_score(yTest, pred)
print("Accuracy:", accuracy)

# confusion matrix
cm = confusion_matrix(yTest, pred)
print("Confusion matrix:\n", cm)

Accuracy: 0.9914129586260734
Confusion matrix:
 [[1291    4]
 [  18 1249]]


In [7]:
text = [
"""
Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hands in the soil and a heart full of passion , Mia transformed an unused corner of the campus into a thriving community garden . She envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate their own fruits and vegetables . Mia believed that this initiative could not only promote environmental consciousness but also foster a sense of community among her peers . As the garden flourished , so did Mia 's connection with her fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables . Mia 's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived , Mia nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of her project resonated deeply , earning her the first prize and a scholarship for further environmental studies . Word of Mia 's success spread , and GreenHaven became a symbol of sustainable living on campus . Mia 's journey didn 't end with the competition ; instead , it marked the beginning of a new chapter . With the scholarship in hand , Mia continued her studies , conducting research on innovative ways to create sustainable urban environments . As Mia graduated from Rivertide University , she left behind a legacy of green initiatives and a campus that had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues . Mia 's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as Mia embarked on her journey beyond college , she carried with her not just a degree but the knowledge that small , meaningful actions could ripple into waves of positive transformation for the planet and its people .
""",
"""
Once upon a time in the vibrant city of Harmonyville ,  lived a college student named   .  was a junior majoring in environmental science at Rivertide University .  love for nature and a deep sense of responsibility towards the planet fueled  determination to make a positive impact . One sunny afternoon ,  stumbled upon a notice about the annual Green Innovation Challenge — an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute ,  decided to develop a sustainable urban gardening initiative called   " GreenHaven . " With  hands in the soil and a heart full of passion ,  transformed an unused corner of the campus into a thriving community garden .  envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate  own fruits and vegetables .  believed that this initiative could not only promote environmental consciousness but also foster a sense of community among  peers . As the garden flourished , so did  's connection with  fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables .  's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived ,  nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of  project resonated deeply , earning  the first prize and a scholarship for further environmental studies . Word of  's success spread , and  became a symbol of sustainable living on campus .  's    t end with the competition ; instead ,  marked the beginning of a new chapter . With the scholarship in hand ,  continued  studies , conducting research on innovative ways to create sustainable urban environments . As  graduated from Rivertide University ,  left behind a legacy of green initiatives and a campus  had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues .  's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as  embarked on  journey beyond college ,  carried with  not just a degree but the knowledge  small , meaningful actions could ripple into waves of positive transformation for the planet and  people .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student named Alex Reynolds . Alex was a junior majoring in computer science at the prestigious Arcadia University . He was a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as Alex was immersed in his studies at the campus library , he stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student . This student was a junior majoring in computer science at the prestigious Arcadia University . They were a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as this student was immersed in their studies at the campus library , they stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the vibrant college town of Crestwood , there lived a spirited young student named Emily . She was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for her boundless enthusiasm , infectious energy , and a love for exploring the world around her . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called  "Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes , she decided to attend their next meeting . As Emily walked into the meeting room , she was greeted by a diverse group of students who shared her passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the Green Harmony team worked tirelessly , planning , planting , and nurturing their garden . Emily found herself forming deep connections with her fellow club members as they faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did Emily 's personal growth . She learned about sustainable practices , organic gardening , and the importance of community engagement . Her once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded them a grant to expand their project . Emily and her friends found themselves at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only her passion for environmental science but also her ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for those around her . As the seasons changed , so did Emily and her friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student who discovered her purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
Once upon a time in the vibrant college town of Crestwood ,  lived a spirited young student named  .  was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for  boundless enthusiasm , infectious energy , and a love for exploring the world around  . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called   " Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes ,  decided to attend  next meeting . As Emily walked into the meeting room ,  was greeted by a diverse group of students  shared  passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the   team worked tirelessly , planning , planting , and nurturing  garden . Emily found  forming deep connections with  fellow club members as  faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did  's personal growth .  learned about sustainable practices , organic gardening , and the importance of community engagement .  once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded  a grant to expand  project . Emily and  friends found  at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only  passion for environmental science but also  ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for  around  . As the seasons changed , so did  and  friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student  discovered  purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
In the vibrant city of Rivertown , there lived a college student named Jason Harris . Jason was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller . He had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper , Jason stumbled upon an article about an elderly woman named Mrs . Eleanor Bennett , who had spent decades working as a librarian in the city . Intrigued by the snippet of her life story , Jason felt compelled to dig deeper and share her tale with the world . After some research , Jason learned that Mrs . Bennett had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However , her own life story had largely remained in the shadows . Determined to uncover the full narrative , Jason reached out to Mrs . Bennett and asked for an interview . To his surprise , she welcomed the opportunity to share her story , and soon they sat down for a conversation that would unveil a lifetime of experiences . As Jason delved into Mrs . Bennett 's past , he discovered a woman who had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown . Her journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative that Jason couldn 't wait to share with his readers . With meticulous attention to detail , Jason crafted a feature article that not only highlighted Mrs . Bennett 's contributions to the community but also captured the essence of her character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs . Bennett became a beloved figure in Rivertown , celebrated for her dedication and the richness of her life story . The success of the article opened doors for Jason , who found himself on the path to becoming a respected journalist . Inspired by Mrs . Bennett 's story , he continued to seek out and share the untold narratives within his community , becoming a storyteller who bridged the gaps between generations . As graduation day approached , Jason reflected on his college journey . His time at Rivertown University had not only provided him with an education but had also instilled in him the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories , Jason ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the vibrant city of Rivertown ,  lived a college student named   .  was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller .  had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper ,  stumbled upon an article about an elderly woman named  .   ,  had spent decades working as a librarian in the city . Intrigued by the snippet of  life story ,  felt compelled to dig deeper and share  tale with the world . After some research ,  learned that  .  had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However ,  own life story had largely remained in the shadows . Determined to uncover the full narrative ,  reached out to  .  and asked for an interview . To  surprise ,  welcomed the opportunity to share  story , and soon  sat down for a conversation  would unveil a lifetime of experiences . As  delved into  .  's past ,  discovered a woman  had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown .  journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative   couldn ' t wait to share with  readers . With meticulous attention to detail ,  crafted a feature article that not only highlighted  .  's contributions to the community but also captured the essence of  character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs .  became a beloved figure in Rivertown , celebrated for  dedication and the richness of  life story . The success of the article opened doors for  ,  found  on the path to becoming a respected journalist . Inspired by  .  's story ,  continued to seek out and share the untold narratives within  community , becoming a storyteller  bridged the gaps between generations . As graduation day approached ,  reflected on  college journey .  time at Rivertown University had not only provided  with an education but had also instilled in  the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories ,  ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the bustling town of Summitville , there lived a college man named Lucas Turner . Lucas was a mechanical engineering major at Summitville Tech , known for his passion for building things from the ground up . His college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with his fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in him . Inspired by the potential of the neglected space , he decided to breathe new life into it and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together they transformed the workshop into the  "Innovation Forge . " It became a haven for students to work on personal projects , exchange ideas , and bring their engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on everything from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and his team decided to organize an annual engineering expo , showcasing the innovative projects that emerged from their workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and his team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for its engineering prowess but also for its potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects that aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ; it was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy he had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner . His college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
""",
"""
In the bustling town of Summitville ,  lived a college man named   . Lucas was a mechanical engineering major at Summitville Tech , known for  passion for building things from the ground up .  college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with  fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in  . Inspired by the potential of the neglected space ,  decided to breathe new life into  and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together  transformed the workshop into the   " Innovation Forge . "  became a haven for students to work on personal projects , exchange ideas , and bring  engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on  from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and  team decided to organize an annual engineering expo , showcasing the innovative projects  emerged from  workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and  team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for  engineering prowess but also for  potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects  aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ;  was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy  had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner .  college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
"""
]
vectorizedText = vectorizer.transform(text)
textsTransformed = pd.DataFrame(vectorizedText.toarray(), columns=vectorizer.get_feature_names_out())
predProb = model.predict_proba(textsTransformed)
pred = model.predict(textsTransformed)

yActual = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1]
explainer = LimeTextExplainer(class_names=['female', 'male'])
for i in range(len(pred)):
    print(i + 1, ":")
    print("Male: ", predProb[i][1])
    print("Female: ", predProb[i][0])
    print("Predicted: ", pred[i])
    if(yActual[i] == 0):
        print("Actual: Female")
    else:
        print("Actual: Male")
        
    vectorized_text = vectorizer.transform([text[i]])
    predict_function = lambda x: model.predict_proba(vectorizer.transform(x))
    explanation = explainer.explain_instance(text[i], predict_function, num_features=20)
    top_words_lime = explanation.as_list()
    print(f"Top words for text response {i + 1}:")
    masculineWords = []
    feminineWords = []
    for word, score in top_words_lime:
        if score > 0:
            masculineWords.append((word, score))
        else:
            feminineWords.append((word, score))
    print("Masculine words: ", masculineWords)
    print("Feminine words: ", feminineWords)
    print("")





1 :
Male:  0.3
Female:  0.7
Predicted:  female
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 1:
Masculine words:  [('an', 0.03821478627938556), ('continued', 0.02978821086354969), ('the', 0.021536330704677097), ('about', 0.02077613256230099), ('believed', 0.016132865042785657), ('on', 0.015965843314523234), ('dedication', 0.015566852528457777), ('of', 0.014942362470522969), ('to', 0.014774066236588806), ('this', 0.014190005465093565), ('journey', 0.01400969322574527), ('peers', 0.013424945182883901), ('arrived', 0.012744391447734113)]
Feminine words:  [('student', -0.046420022083196455), ('her', -0.04507398407878916), ('Her', -0.04046442432692137), ('she', -0.02567286502479722), ('students', -0.023033446407819423), ('She', -0.022316484109798314), ('story', -0.00543503092442319)]

2 :
Male:  0.41
Female:  0.59
Predicted:  female
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 2:
Masculine words:  [('an', 0.029231522551867468), ('the', 0.02322746599301921), ('about', 0.018860698565750497), ('believed', 0.01736505680378248), ('in', 0.015377257213985016), ('peers', 0.012353890492505726), ('vision', 0.010795475341835828), ('to', 0.010588569543289347), ('of', 0.010438915383223129), ('full', 0.009680190837781), ('arrived', 0.009220302642446571)]
Feminine words:  [('student', -0.08190183875091495), ('students', -0.05705552989874296), ('that', -0.02148079001453864), ('only', -0.013904812598570977), ('story', -0.011114995921748909), ('had', -0.010521485433941092), ('community', -0.008190026924886297), ('ideas', -0.008164221328556372), ('graduated', -0.007843761843137715)]

3 :
Male:  0.67
Female:  0.33
Predicted:  male
Actual: Male


...

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 8:
Masculine words:  [('children', 0.02003036059552191), ('about', 0.01683972089205913), ('After', 0.014271415721131617), ('also', 0.013257149669166043), ('into', 0.012214233135427483), ('of', 0.012160852164215226), ('life', 0.011817202081271082), ('contributions', 0.011576972301587144), ('remained', 0.011183901468066642), ('full', 0.010462987372514915)]
Feminine words:  [('journalist', -0.05074438493308702), ('student', -0.03862205966969275), ('Mrs', -0.026962592893471825), ('librarian', -0.020885916446170955), ('experiences', -0.01893365126957383), ('stories', -0.012502860323011275), ('character', -0.012462457922825748), ('article', -0.012299721068286128), ('human', -0.009373454844839318), ('woman', -0.008288593195103089)]

9 :
Male:  0.66
Female:  0.34
Predicted:  male
Actual: Male


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 9:
Masculine words:  [('his', 0.04166599360734337), ('His', 0.040848597098544256), ('he', 0.039922814668908395), ('One', 0.01827785587433772), ('team', 0.015054051655480398), ('of', 0.013614997155296519), ('him', 0.013614575879201589), ('design', 0.010042262806185328), ('like', 0.009965362884229379), ('personal', 0.009078865303580696), ('was', 0.008126417340934434), ('from', 0.007757405824712373)]
Feminine words:  [('students', -0.06119627258174772), ('experience', -0.04953973379274483), ('had', -0.01873470759895846), ('old', -0.01606002500683525), ('night', -0.015170282914241695), ('sight', -0.013560185447673291), ('talent', -0.013282856600094421), ('ideas', -0.012606096064806194)]

10 :
Male:  0.55
Female:  0.45
Predicted:  male
Actual: Male
Top words for text response 10:
Masculine words:  [('of', 0.023724967287013402), ('design', 0.022118727083902768), ('was', 0.01695305178928676), ('building', 0.015492976640949145), ('about', 0.014170806601243719), ('be

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


XGBoost

In [8]:
# load data
dataset = pd.read_csv('../datasets/BUG/balanced_BUG.csv')
dataset.replace("male", 1, inplace=True)
dataset.replace("female", 0, inplace=True)
print(dataset.head())
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
# get samples with only neutral or stereotype sentence
train = train[train['stereotype'].isin([0, 1])]
test = test[test['stereotype'].isin([0, 1])]

vectorizer = CountVectorizer()
trainTexts = vectorizer.fit_transform(train['sentence_text'])
testTexts = vectorizer.transform(test['sentence_text'])

xTrain = pd.DataFrame(trainTexts.toarray(), columns=vectorizer.get_feature_names_out())
xTest = pd.DataFrame(testTexts.toarray(), columns=vectorizer.get_feature_names_out())
yTrain = train['predicted gender']
yTest = test['predicted gender']

# train model
model = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
model.fit(xTrain, yTrain, eval_set=[(xTest, yTest)])

with open('../savedModels/XGBoostModel.pkl', 'wb') as model_file:
    pickle.dump((model, vectorizer), model_file)

   Unnamed: 0                                      sentence_text  \
0           0  Notice of contact depends on the time the infe...   
1           1  Hepaprotective agent glycyrrhizinate was added...   
2           2  So yes , that was humbling . " Addressing the ...   
3           3  Following previous works based on classical pr...   
4           4  The second aspect is the agent neighborhood ! ...   

                                              tokens profession    g  \
0  ['Notice', 'of', 'contact', 'depends', 'on', '...     person  her   
1  ['Hepaprotective', 'agent', 'glycyrrhizinate',...      agent  her   
2  ['So', 'yes', ',', 'that', 'was', 'humbling', ...  assistant  her   
3  ['Following', 'previous', 'works', 'based', 'o...      agent  her   
4  ['The', 'second', 'aspect', 'is', 'the', 'agen...      agent  her   

   profession_first_index  g_first_index  predicted gender  stereotype  \
0                       9             13                 0           1   
1         

In [9]:
# accuracy
pred = model.predict(xTest)

accuracy = accuracy_score(yTest, pred)
print("Accuracy:", accuracy)

# confusion matrix
cm = confusion_matrix(yTest, pred)
print("Confusion matrix:\n", cm)

Accuracy: 0.9898516783762685
Confusion matrix:
 [[1282   13]
 [  13 1254]]


In [10]:
text = [
"""
Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hands in the soil and a heart full of passion , Mia transformed an unused corner of the campus into a thriving community garden . She envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate their own fruits and vegetables . Mia believed that this initiative could not only promote environmental consciousness but also foster a sense of community among her peers . As the garden flourished , so did Mia 's connection with her fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables . Mia 's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived , Mia nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of her project resonated deeply , earning her the first prize and a scholarship for further environmental studies . Word of Mia 's success spread , and GreenHaven became a symbol of sustainable living on campus . Mia 's journey didn 't end with the competition ; instead , it marked the beginning of a new chapter . With the scholarship in hand , Mia continued her studies , conducting research on innovative ways to create sustainable urban environments . As Mia graduated from Rivertide University , she left behind a legacy of green initiatives and a campus that had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues . Mia 's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as Mia embarked on her journey beyond college , she carried with her not just a degree but the knowledge that small , meaningful actions could ripple into waves of positive transformation for the planet and its people .
""",
"""
Once upon a time in the vibrant city of Harmonyville ,  lived a college student named   .  was a junior majoring in environmental science at Rivertide University .  love for nature and a deep sense of responsibility towards the planet fueled  determination to make a positive impact . One sunny afternoon ,  stumbled upon a notice about the annual Green Innovation Challenge — an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute ,  decided to develop a sustainable urban gardening initiative called   " GreenHaven . " With  hands in the soil and a heart full of passion ,  transformed an unused corner of the campus into a thriving community garden .  envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate  own fruits and vegetables .  believed that this initiative could not only promote environmental consciousness but also foster a sense of community among  peers . As the garden flourished , so did  's connection with  fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables .  's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived ,  nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of  project resonated deeply , earning  the first prize and a scholarship for further environmental studies . Word of  's success spread , and  became a symbol of sustainable living on campus .  's    t end with the competition ; instead ,  marked the beginning of a new chapter . With the scholarship in hand ,  continued  studies , conducting research on innovative ways to create sustainable urban environments . As  graduated from Rivertide University ,  left behind a legacy of green initiatives and a campus  had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues .  's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as  embarked on  journey beyond college ,  carried with  not just a degree but the knowledge  small , meaningful actions could ripple into waves of positive transformation for the planet and  people .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student named Alex Reynolds . Alex was a junior majoring in computer science at the prestigious Arcadia University . He was a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as Alex was immersed in his studies at the campus library , he stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student . This student was a junior majoring in computer science at the prestigious Arcadia University . They were a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as this student was immersed in their studies at the campus library , they stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the vibrant college town of Crestwood , there lived a spirited young student named Emily . She was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for her boundless enthusiasm , infectious energy , and a love for exploring the world around her . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called  "Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes , she decided to attend their next meeting . As Emily walked into the meeting room , she was greeted by a diverse group of students who shared her passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the Green Harmony team worked tirelessly , planning , planting , and nurturing their garden . Emily found herself forming deep connections with her fellow club members as they faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did Emily 's personal growth . She learned about sustainable practices , organic gardening , and the importance of community engagement . Her once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded them a grant to expand their project . Emily and her friends found themselves at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only her passion for environmental science but also her ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for those around her . As the seasons changed , so did Emily and her friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student who discovered her purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
Once upon a time in the vibrant college town of Crestwood ,  lived a spirited young student named  .  was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for  boundless enthusiasm , infectious energy , and a love for exploring the world around  . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called   " Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes ,  decided to attend  next meeting . As Emily walked into the meeting room ,  was greeted by a diverse group of students  shared  passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the   team worked tirelessly , planning , planting , and nurturing  garden . Emily found  forming deep connections with  fellow club members as  faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did  's personal growth .  learned about sustainable practices , organic gardening , and the importance of community engagement .  once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded  a grant to expand  project . Emily and  friends found  at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only  passion for environmental science but also  ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for  around  . As the seasons changed , so did  and  friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student  discovered  purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
In the vibrant city of Rivertown , there lived a college student named Jason Harris . Jason was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller . He had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper , Jason stumbled upon an article about an elderly woman named Mrs . Eleanor Bennett , who had spent decades working as a librarian in the city . Intrigued by the snippet of her life story , Jason felt compelled to dig deeper and share her tale with the world . After some research , Jason learned that Mrs . Bennett had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However , her own life story had largely remained in the shadows . Determined to uncover the full narrative , Jason reached out to Mrs . Bennett and asked for an interview . To his surprise , she welcomed the opportunity to share her story , and soon they sat down for a conversation that would unveil a lifetime of experiences . As Jason delved into Mrs . Bennett 's past , he discovered a woman who had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown . Her journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative that Jason couldn 't wait to share with his readers . With meticulous attention to detail , Jason crafted a feature article that not only highlighted Mrs . Bennett 's contributions to the community but also captured the essence of her character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs . Bennett became a beloved figure in Rivertown , celebrated for her dedication and the richness of her life story . The success of the article opened doors for Jason , who found himself on the path to becoming a respected journalist . Inspired by Mrs . Bennett 's story , he continued to seek out and share the untold narratives within his community , becoming a storyteller who bridged the gaps between generations . As graduation day approached , Jason reflected on his college journey . His time at Rivertown University had not only provided him with an education but had also instilled in him the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories , Jason ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the vibrant city of Rivertown ,  lived a college student named   .  was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller .  had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper ,  stumbled upon an article about an elderly woman named  .   ,  had spent decades working as a librarian in the city . Intrigued by the snippet of  life story ,  felt compelled to dig deeper and share  tale with the world . After some research ,  learned that  .  had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However ,  own life story had largely remained in the shadows . Determined to uncover the full narrative ,  reached out to  .  and asked for an interview . To  surprise ,  welcomed the opportunity to share  story , and soon  sat down for a conversation  would unveil a lifetime of experiences . As  delved into  .  's past ,  discovered a woman  had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown .  journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative   couldn ' t wait to share with  readers . With meticulous attention to detail ,  crafted a feature article that not only highlighted  .  's contributions to the community but also captured the essence of  character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs .  became a beloved figure in Rivertown , celebrated for  dedication and the richness of  life story . The success of the article opened doors for  ,  found  on the path to becoming a respected journalist . Inspired by  .  's story ,  continued to seek out and share the untold narratives within  community , becoming a storyteller  bridged the gaps between generations . As graduation day approached ,  reflected on  college journey .  time at Rivertown University had not only provided  with an education but had also instilled in  the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories ,  ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the bustling town of Summitville , there lived a college man named Lucas Turner . Lucas was a mechanical engineering major at Summitville Tech , known for his passion for building things from the ground up . His college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with his fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in him . Inspired by the potential of the neglected space , he decided to breathe new life into it and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together they transformed the workshop into the  "Innovation Forge . " It became a haven for students to work on personal projects , exchange ideas , and bring their engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on everything from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and his team decided to organize an annual engineering expo , showcasing the innovative projects that emerged from their workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and his team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for its engineering prowess but also for its potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects that aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ; it was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy he had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner . His college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
""",
"""
In the bustling town of Summitville ,  lived a college man named   . Lucas was a mechanical engineering major at Summitville Tech , known for  passion for building things from the ground up .  college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with  fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in  . Inspired by the potential of the neglected space ,  decided to breathe new life into  and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together  transformed the workshop into the   " Innovation Forge . "  became a haven for students to work on personal projects , exchange ideas , and bring  engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on  from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and  team decided to organize an annual engineering expo , showcasing the innovative projects  emerged from  workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and  team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for  engineering prowess but also for  potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects  aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ;  was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy  had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner .  college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
"""
]
vectorizedText = vectorizer.transform(text)
textsTransformed = pd.DataFrame(vectorizedText.toarray(), columns=vectorizer.get_feature_names_out())
predProb = model.predict_proba(textsTransformed)
pred = model.predict(textsTransformed)

yActual = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1]
explainer = LimeTextExplainer(class_names=['female', 'male'])
for i in range(len(pred)):
    print(i + 1, ":")
    print("Male: ", predProb[i][1])
    print("Female: ", predProb[i][0])
    print("Predicted: ", pred[i])
    if(yActual[i] == 0):
        print("Actual: Female")
    else:
        print("Actual: Male")
        
    vectorized_text = vectorizer.transform([text[i]])
    predict_function = lambda x: model.predict_proba(vectorizer.transform(x))
    explanation = explainer.explain_instance(text[i], predict_function, num_features=20)
    top_words_lime = explanation.as_list()
    print(f"Top words for text response {i + 1}:")
    masculineWords = []
    feminineWords = []
    for word, score in top_words_lime:
        if score > 0:
            masculineWords.append((word, score))
        else:
            feminineWords.append((word, score))
    print("Masculine words: ", masculineWords)
    print("Feminine words: ", feminineWords)
    print("")

1 :
Male:  0.00072941085
Female:  0.9992706
Predicted:  0
Actual: Female
Top words for text response 1:
Masculine words:  [('her', 0.008910332115293943), ('majoring', 0.00031592683840320484), ('arrived', 0.0003149875485987437), ('prize', 0.0002781893700498528), ('research', 0.0002559216013681642), ('end', 0.00024312402786462692), ('learn', 0.00021890467776286356), ('day', 0.0002063095179815171), ('hope', 0.00019712604591185857)]
Feminine words:  [('Her', -0.005097616801398266), ('stumbled', -0.00027814548505274893), ('Green', -0.00023975182757048904), ('deeply', -0.00023164691710813714), ('green', -0.00022086300650129437), ('sustainability', -0.00019763144627554598), ('thrive', -0.00019723399984557818), ('towards', -0.00017996467410795458), ('judges', -0.00017015532366415834), ('corner', -0.00016591823449057743), ('garden', -0.00016583594547507502)]

2 :
Male:  0.9931248
Female:  0.006875217
Predicted:  1
Actual: Female
Top words for text response 2:
Masculine words:  [('degree', 6.105